In [1]:

import sys
sys.path.append('/Users/johnlawson/PycharmProjects/Herbie/herbie/models/aqm.py')  # Adjust this path to your Herbie installation


from herbie import Herbie
from datetime import datetime, timedelta

# Yesterday's date
date = datetime.utcnow() - timedelta(days=1)

print(f"Testing AQM with date {date}")

# Create Herbie object
H = Herbie(
    date,
    model="aqm",
    product="ozone",
    run=12,
    fxx=1,
    verbose=True,
)

# Check file availability
print("File sources available:", H.check_file_availability())

# Try to download
try:
    local_file = H.download()
    print(f"Successfully downloaded to {local_file}")

    # Check inventory
    inv = H.inventory()
    print("Inventory preview:")
    print(inv.head())

    # Read data
    ds = H.xarray("OZCON")
    print("\nDataset info:")
    print(ds)
except Exception as e:
    print(f"Error: {e}")


Testing AQM with date 2025-04-09 04:55:02.721479


KeyError: 'ozone'

In [ ]:
assert 1==0

In [ ]:
# AQM Ozone Data Example
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from datetime import datetime, timedelta

import sys
sys.path.append('/Users/johnlawson/PycharmProjects/Herbie/herbie/models/aqm.py')  # Adjust this path to your Herbie installation
from herbie import Herbie

# Set up the notebook for plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

# Create a Herbie object for AQM ozone data
# Let's get yesterday's 12Z run, 24-hour forecast

yesterday = datetime(2025, 4, 4)
H = Herbie(
    f"{yesterday.strftime('%Y%m%d')} 12:00",
    model="aqm",
    product="ozone",
    fxx=0,
)

In [ ]:
print(f"URL: {H.url}")

# Display information about the file
print(f"Model: {H.model}")
print(f"Date: {H.date}")
print(f"Run: {H.run}")
print(f"Forecast hour: {H.fxx}")
print(f"Product: {H.product}")

# Check if the file exists and show the URL
print(f"URL: {H.get_url()}")

# Look at the inventory of the file
print("\nFile Inventory:")
inventory = H.inventory()
print(inventory.head(10))  # Show first 10 entries

# Download the file
local_file = H.download()
print(f"\nDownloaded to: {local_file}")

# Read ozone data with xarray
# Typically, surface ozone is what we're interested in
ds = H.xarray("OZCON:1 hybrid level")

# Display dataset information
print("\nDataset Information:")
print(ds)

# Plot the data
plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.LambertConformal(
    central_longitude=-97.5, central_latitude=38.5))

# Add map features
ax.coastlines()
ax.add_feature(ccrs.cartopy.feature.STATES)

# Plot the ozone concentration
cf = ds["OZCON_1hybridlevel"].plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='jet',
    vmin=0,
    vmax=80,
    add_colorbar=False
)

# Add colorbar
cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05, aspect=50)
cbar.set_label('Ozone Concentration (ppb)')

# Add title
plt.title(f'AQM Ozone Forecast\n{H.date.strftime("%Y-%m-%d")} {H.run:02d}Z +{H.fxx}h', fontsize=16)

plt.tight_layout()
plt.show()

# Extract data at a specific location (e.g., New York City)
nyc_lat, nyc_lon = 40.7128, -74.0060
print(f"\nOzone concentration near New York City ({nyc_lat}, {nyc_lon}):")
point_data = ds.herbie.nearest_point(nyc_lat, nyc_lon)
print(f"Value: {point_data['OZCON_1hybridlevel'].values} ppb")
